<a href="https://colab.research.google.com/github/mrhamedani/Deep-learning-projects-Tensorflow/blob/main/8_fashion_mnist_hypertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# hypertuning
Parameters such as weights and biases improve during training. But we have to determine the best hyperparameters ourselves. Such as **the number of hidden layers**, **batch-size**, **learning rate**, **the number of neurons in each layer**, **activate function**, and **epochs**.

#hyperband
There are several methods to determine the best hyperparameters, the best of which is hyperband, which we will review here

In [ ]:
! pip install keras-tuner

In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### hyperparameters tune --> 1-number of neurons    2-learning rate

In [8]:
def model_builder(hp):       # hp = hyperparameters # documentation https://www.tensorflow.org/tutorials/keras/keras_tuner
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  #tune number of units
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32) # number of neurons = hp_units ,between 32 and 512 # type of data = int step = 32
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))
  # tune learning rate can be 1e-2, 1e-3, 1e-4
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy', # val_accuracy  or val_loss determine measures for best hyperparameters
                     max_epochs=10,    # maximum number of epochs for each hyperparameter combination
                     factor=3, # factor = 3 means 3x3 = 9 hyperparameter combinations
                     directory='my_dir', # for saving log
                     project_name='reza')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)  # stop training if no improvement in 5 epochs

In [14]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early]) # search for best hyperparameters in 50 epochs (maximum)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Learning Rate: {best_hps.get('learning_rate')}")
print(f"Best Units: {best_hps.get('units')}")

Best Learning Rate: 0.001
Best Units: 352


# hyperparameters tune --> epoch

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc)) + 1
print(f"Best epoch: {best_epoch}")            # best epochs  = 18

model = tuner.hypermodel.build(best_hps)      # fit model with best epochs=18  & best hyperparameters
model.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

In [16]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8542 - loss: 0.4128
[test loss, test accuracy]: [0.42475688457489014, 0.8493000268936157]
